# Database Connection Test
This program attempts to guess the correct database connection string based on your computer's platform.
It then tests whether a successful connection can be made to the chosen database, and if a simple select statement can return data from the chosen table.

## Input

In [ ]:
db = 'MuesliAG'
table = 'Materials'

### Libraries

In [ ]:
import platform
import warnings
import pyodbc
import sqlalchemy as sal
import pandas as pd

### Define Functions

In [ ]:
def db_connect_str_from_env (_db, _driver = None):
    template = 'mssql+pyodbc:///?odbc_connect=DRIVER={};SERVER=dlyle.database.windows.net;DATABASE={};UID=student;PWD=Viz(Data);'

    if not _driver is None:
        return template.format(_driver, _db)

    print('Guessing driver based on', platform.system(), platform.release(), platform.machine(), platform.platform())

    if platform.system() == 'Darwin': #MacOS
        if platform.machine() == 'arm64': #M1 chip
            driver = '/opt/homebrew/lib/libmsodbcsql.18.dylib'
        else:
            driver = '/Library/simba/sqlserverodbc/lib/libsqlserverodbc_sbu.dylib'
    else: #Windows and anything else
        driver = '{SQL Server}'
    
    return template.format(driver, _db)

In [ ]:
def data_test (_c_str, _tbl):
    try:
        cxn = sal.create_engine(_c_str).connect()
        print(_c_str)
        print('Success!')
        data = pd.read_sql(f"select TOP 5 * from {_tbl}", cxn)
        cxn.close()
        return data
    except Exception as err:
        warnings.warn(str(err))
        return None

## Connection Tests
#### First, let's try to guess connection string based on environment

If this next block returns data, you can stop here. Use the connection string shown on the line above the data.

In [ ]:
data_test(db_connect_str_from_env(db),table)

#### Otherwise, try out some other connection strings

##### Generic, non version specific SQL driver

In [ ]:
data_test(db_connect_str_from_env(db, '{SQL Server}'),table)

##### Version specific SQL driver
You can also try versions 15, 13 or 11 if 17 is not available (for some reason Microsoft likes odd version numbers...)

In [ ]:
data_test(db_connect_str_from_env(db, '{ODBC Driver 17 for SQL Server}'),table)

##### For non Windows, refer to the driver's library file directly
Simba driver provided by Tableau

In [ ]:
data_test(db_connect_str_from_env(db, '/Library/simba/sqlserverodbc/lib/libsqlserverodbc_sbu.dylib'),table)

Microsoft drivers - can also try older versions (17, 15, 13, 11 etc.)

In [ ]:
data_test(db_connect_str_from_env(db, '/opt/homebrew/lib/libmsodbcsql.18.dylib'),table)

In [ ]:
data_test(db_connect_str_from_env(db, '/usr/local/lib/libmsodbcsql.18.dylib'),table)